In [3]:
import sqlite3
import pandas as pd

conn = sqlite3.connect("business_analyst.db")
cursor = conn.cursor()

In [5]:
file_path = "BusinessAnalyst.csv"

try:
    df = pd.read_csv(file_path)
    print("File loaded sucessfully!")
    print(df.head())
except Exception as e:
    print(f"Error loading file: {e}")

File loaded sucessfully!
  Unnamed: 0 index                                         Job Title  \
0          0     0  Business Analyst - Clinical & Logistics Platform   
1          1     1                                  Business Analyst   
2          2     2                                      Data Analyst   
3          3     3   Information Security Analyst, Incident Response   
4          4     4                     Analyst - FP&A Global Revenue   

               Salary Estimate  \
0  $56K-$102K (Glassdoor est.)   
1  $56K-$102K (Glassdoor est.)   
2  $56K-$102K (Glassdoor est.)   
3  $56K-$102K (Glassdoor est.)   
4  $56K-$102K (Glassdoor est.)   

                                     Job Description Rating  \
0  Company Overview\n\n\nAt Memorial Sloan Ketter...    3.9   
1  We are seeking for an energetic and collaborat...    3.8   
2  For more than a decade, Asembia has been worki...    3.6   
3  Job Description Summary\nThe Information Secur...    3.6   
4  Magnite is the worl

In [9]:
cursor.execute("""
CREATE TABLE IF NOT EXISTS BusinessAnalyst (
    Unique_ID VARCHAR(255) PRIMARY KEY,
    Job_Title VARCHAR(255) NOT NULL,
    Company_Name VARCHAR(255) NOT NULL,
    Location VARCHAR(255) NOT NULL,
    SalaryEstimate VARCHAR(50),
    Rating DECIMAL(3,2),
    Founded INTEGER CHECK (Founded > 0),
    Industry VARCHAR(255),
    Sector VARCHAR(255),
    Revenue VARCHAR(100),
    Easy_Apply BOOLEAN
)
""")

In [11]:
cursor.execute("""
CREATE TABLE IF NOT EXISTS Company (
    Company_ID INTEGER PRIMARY KEY AUTOINCREMENT,
    Company_Name VARCHAR(255) UNIQUE NOT NULL,
    Industry VARCHAR(255),
    Sector VARCHAR(255),
    Revenue VARCHAR(100),
    Founded INTEGER CHECK (Founded > 0),
    Rating DECIMAL(3,2)
)
""")

In [13]:
cursor.execute("""
CREATE TABLE IF NOT EXISTS Jobs (
    Job_ID INTEGER PRIMARY KEY AUTOINCREMENT,
    Unique_ID VARCHAR(255) UNIQUE NOT NULL,
    Job_Title VARCHAR(255) NOT NULL,
    Company_ID INTEGER,
    Location VARCHAR(255) NOT NULL,
    Salary_Estimate VARCHAR(50),
    Easy_Apply BOOLEAN,
    FOREIGN KEY (Company_ID) REFERENCES Company(Company_ID) ON DELETE CASCADE
)
""")

In [15]:
conn.commit()
conn.close()

In [17]:
index_unique = df["index"].is_unique
index_null = df["index"].isnull().sum()

print(f"Is `index` a Unique Primary Key?: {index_unique}")
print(f"Missing Values in `index` Column: {index_null}")

Is `index` a Unique Primary Key?: False
Missing Values in `index` Column: 0


In [19]:
print(f"Unique Job Titles: {df['Job Title'].nunique()}")
print(f"Unique Company Names: {df['Company Name'].nunique()}")
print(f"Unique Locations: {df['Location'].nunique()}")

Unique Job Titles: 2210
Unique Company Names: 2197
Unique Locations: 218


In [21]:
duplicate_rows = df.duplicated().sum()
print(f"Total Duplicate Rows: {duplicate_rows}")

Total Duplicate Rows: 0


In [23]:
unique_combinations = df[['Job Title', 'Company Name', 'Location']].drop_duplicates().shape[0]
total_rows = df.shape[0]

print(f"Unique (Job Title, Company Name, Location) combinations: {unique_combinations}")
print(f"Total Rows in Dataset: {total_rows}")

Unique (Job Title, Company Name, Location) combinations: 4035
Total Rows in Dataset: 4092


In [25]:
df["Unique_ID"] = df["Job Title"] + "_" + df["Company Name"] + "_" + df["Location"] + "_" + df.index.astype(str)
print(f"Unique Identifiers: {df['Unique_ID'].nunique()} / {df.shape[0]}")

Unique Identifiers: 4092 / 4092


In [27]:
company_counts = df["Company Name"].value_counts()
single_entry_companies = company_counts[company_counts == 1]

print(f"Number of companies with only one job posting: {len(single_entry_companies)}")

Number of companies with only one job posting: 1575


In [29]:
print("\nData Types:\n", df.dtypes)
df["Rating"] = pd.to_numeric(df["Rating"], errors="coerce")
df["Founded"] = pd.to_numeric(df["Founded"], errors="coerce")
df["Easy Apply"] = df["Easy Apply"].astype(bool)


Data Types:
 Unnamed: 0           object
index                object
Job Title            object
Salary Estimate      object
Job Description      object
Rating               object
Company Name         object
Location             object
Headquarters         object
Size                 object
Founded              object
Type of ownership    object
Industry             object
Sector               object
Revenue              object
Competitors          object
Easy Apply           object
Unique_ID            object
dtype: object


In [31]:
invalid_ratings = df[(df["Rating"] < 0) | (df["Rating"] > 5)]
print(f"\nInvalid Ratings Count: {len(invalid_ratings)}")


Invalid Ratings Count: 318


In [33]:
invalid_founded = df[(df["Founded"] < 1600) | (df["Founded"] > 2025)]
print(f"\nInvalid Founded Years Count: {len(invalid_founded)}")


Invalid Founded Years Count: 1006


In [35]:
def detect_outliers(column):
    Q1 = df[column].quantile(0.25)
    Q3 = df[column].quantile(0.75)
    IQR = Q3 - Q1
    return df[(df[column] < (Q1 - 1.5 * IQR)) | (df[column] > (Q3 + 1.5 * IQR))]

rating_outliers = detect_outliers("Rating")
print(f"\nNumber of Outliers in Rating: {len(rating_outliers)}")


Number of Outliers in Rating: 333


In [37]:
missing_values = df.isnull().sum()
print("Missing Values:\n", missing_values)

Missing Values:
 Unnamed: 0             0
index                  0
Job Title              0
Salary Estimate        0
Job Description        0
Rating               400
Company Name           0
Location               0
Headquarters           0
Size                   0
Founded              341
Type of ownership      0
Industry               0
Sector                 0
Revenue                0
Competitors          400
Easy Apply             0
Unique_ID              0
dtype: int64


In [41]:
df["Rating"] = df["Rating"].fillna(df["Rating"].median())

In [43]:
df["Founded"] = df["Founded"].fillna(df["Founded"].median())

In [45]:
df["Competitors"] = df["Competitors"].fillna("No Competitors")

In [47]:
missing_values = df.isnull().sum()
print("Missing Values:\n", missing_values)

Missing Values:
 Unnamed: 0           0
index                0
Job Title            0
Salary Estimate      0
Job Description      0
Rating               0
Company Name         0
Location             0
Headquarters         0
Size                 0
Founded              0
Type of ownership    0
Industry             0
Sector               0
Revenue              0
Competitors          0
Easy Apply           0
Unique_ID            0
dtype: int64


In [49]:
import re

def clean_company_name(name):
    return re.sub(r"\s\d+(\.\d+)?$", "", name)

df["Company Name"] = df["Company Name"].apply(clean_company_name)

print(df["Company Name"].head())

0    Memorial Sloan-Kettering
1     Paine Schwartz Partners
2                     Asembia
3                          BD
4             Rubicon Project
Name: Company Name, dtype: object


In [53]:
cleaned_file_path = "BusinessAnalyst_Cleaned.csv"
df.to_csv(cleaned_file_path, index=False)
print(f"✅ Cleaned dataset saved as {cleaned_file_path}")

✅ Cleaned dataset saved as BusinessAnalyst_Cleaned.csv
